In [1]:
import copy
import numpy as np

def parse_board(board):
    """
    Converts a string representation of a Sudoku board into a numpy array, 
    handling various input formats and converting non-digits to zeros.
    
    Args:
        board (str): A string representing a Sudoku board where:
            - Rows are separated by newlines
            - Numbers can be separated by spaces (which are ignored)
            - Non-digit characters are treated as empty cells
            - Each row should contain 9 valid positions after parsing
    
    Returns:
        numpy.ndarray: A 9x9 numpy array where:
            - Digits from input are converted to integers
            - Non-digit characters are converted to 0
            - Spaces are ignored
    
    Process:
        1. Strips leading/trailing whitespace from input string
        2. Splits string into rows at newlines
        3. For each row:
            - Strips whitespace
            - Filters out spaces
            - Converts digits to integers
            - Converts non-digits to 0
        4. Converts resulting list of lists to numpy array
    
    Example:
        Input:
            "1 2 3\n
             . . 4\n
             7 8 9"
        Output:
            array([[1, 2, 3],
                  [0, 0, 4],
                  [7, 8, 9]])
    """
    return np.array([[int(c) if c.isdigit() else 0 for c in row.strip() if c != " "] for row in board.strip().split('\n')])

def print_board(board):
    """
    Prints a Sudoku board in a formatted, readable grid layout with 
    dividing lines to clearly show 3x3 boxes.
    
    Args:
        board (list/ndarray): A 9x9 2D array/list representing a Sudoku board
    
    Output Format:
        - Horizontal lines (- - - - - - - - - - - -) separate 3x3 box rows
        - Vertical bars (|) separate 3x3 box columns
        - Numbers are separated by spaces
        - Each row ends with a newline
        - Extra newline at the end for spacing
    
    Example Output:
        1 2 3 | 4 5 6 | 7 8 9
        4 5 6 | 7 8 9 | 1 2 3
        7 8 9 | 1 2 3 | 4 5 6
        - - - - - - - - - - - -
        2 3 4 | 5 6 7 | 8 9 1
        5 6 7 | 8 9 1 | 2 3 4
        8 9 1 | 2 3 4 | 5 6 7
        - - - - - - - - - - - -
        3 4 5 | 6 7 8 | 9 1 2
        6 7 8 | 9 1 2 | 3 4 5
        9 1 2 | 3 4 5 | 6 7 8
    """
    for i, row in enumerate(board):
        if i % 3 == 0 and i != 0:
            print("- - - - - - - - - - - -")
        for j, val in enumerate(row):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")
            if j == 8:
                print(val)
            else:
                print(str(val) + " ", end="")
    print()


def board_is_solved(board):
    """
    Validates whether a Sudoku board is completely and correctly solved according to all Sudoku rules.
    
    Args:
        board (list): A 9x9 2D list representing a filled Sudoku board
    
    Returns:
        bool: True if the board is a valid solution, False otherwise
    
    Validation Process:
        For each number (1-9), checks three types of constraints:
        1. Row Constraints:
            - Each number must appear exactly once in each row
            - Returns False if number is missing or appears multiple times
        
        2. Column Constraints:
            - Each number must appear exactly once in each column
            - Returns False if number is missing or appears multiple times
        
        3. 3x3 Box Constraints:
            - Each number must appear exactly once in each 3x3 box
            - Returns False if number is missing or appears multiple times
    
    Examples:
        Valid board will contain:
        - Each row: exactly one of each number 1-9
        - Each column: exactly one of each number 1-9
        - Each 3x3 box: exactly one of each number 1-9
    
    Note:
        - Returns False immediately upon finding any violation
        - Must pass all constraints for all numbers to return True
        - Assumes board is completely filled (no zeros/empty cells)
        - Time complexity is O(n^3) where n=9 (fixed for Sudoku)
    """

    for num in range(1, 10):
        # Check rows
        for i in range(9):
            count = 0
            for j in range(9): #cycle through columns of a fixed row
                if board[i][j] == num:
                    count += 1
                if count > 1: #if a number appear more than once in a row
                    return False
            if count < 1: #if a number doesn't appear in a row
                return False
                    
        # check columns
        for j in range(9):
            count = 0
            for i in range(9): #cycle through rows of a fixed column
                if board[i][j] == num:
                    count += 1
                if count > 1: #if a number appear more than once in a row
                    return False
            if count < 1: #if a number doesn't appear in a row
                return False

    # Check boxes
        for box_x in range(1, 3):
            for box_y in range(1, 3):
                count = 0
                for i in range(box_y * 3, box_y * 3 + 3):
                    for j in range(box_x * 3, box_x * 3 + 3):
                        if board[i][j] == num:
                            count += 1
                        if(count > 1): #if a number appears more than one time in a box 3x3
                            return False
                if count < 1: #if a number doesn't appear in a box
                    return False
    return True


def board_is_full(board):
    """
    Checks if a Sudoku board is completely filled (contains no empty cells).
    
    Args:
        board (list): A 9x9 2D list representing a Sudoku board where:
            - 0 represents an empty cell
            - Numbers 1-9 represent filled cells
    
    Returns:
        bool: True if no empty cells (zeros) exist, False otherwise
    
    Process:
        1. Iterates through each cell in the 9x9 grid
        2. Returns False immediately upon finding any zero
        3. Returns True only if no zeros are found
    
    Example:
        >>> board = [[5,3,0],  # Contains zero -> False
                    [6,0,9],
                    [8,9,2]]
        >>> board = [[5,3,4],  # No zeros -> True
                    [6,7,9],
                    [8,9,2]]
    """
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                return False
    return True

In [2]:
import random
from multiprocessing.pool import ThreadPool as Pool

def generate_board_state(initial_board):
    """
    Generates a complete Sudoku board state from a partially filled board by randomly filling empty cells 
    while respecting row constraints.
    
    Args:
        initial_board (list): A 9x9 2D list representing a partially filled Sudoku board where:
            - 0 represents an empty cell
            - Numbers 1-9 represent fixed cells that shouldn't be modified
    
    Returns:
        list: A 9x9 2D list representing a complete board state where:
            - All original numbers from initial_board are preserved in their positions
            - All empty cells (0s) are filled with random numbers
            - Each row contains all numbers from 1-9 exactly once
    
    Process:
        1. Creates a deep copy of initial board to avoid modifying the original
        2. For each row:
            - Identifies which numbers (1-9) are missing from that row
            - Shuffles these missing numbers randomly
            - Fills empty cells in the row with the shuffled numbers
    
    Note:
        - Maintains the initial board's constraints (doesn't modify filled cells)
        - Ensures row-wise uniqueness of numbers 1-9
        - Does not guarantee column or 3x3 box constraints are met
    """
    
    initial = copy.deepcopy(initial_board)
    for i in range(9):
        gene = list()
        # create a list for every row with the numbers from 1 to 9, removing the numbers already present in the row
        for j in range(1,10):
            if not j in initial_board[i]:
                gene.append(j)
        random.shuffle(gene)
        
        for j in range(9):
            if initial_board[i][j] != 0:
                initial[i][j] = initial_board[i][j]
            else:
                initial[i][j] = gene.pop()
    
    return initial


def coords_already_full(board):
    """
    Finds all positions in a Sudoku board that are already filled with numbers (non-zero values).
    
    Args:
        board (list): A 9x9 2D list representing the Sudoku board, where:
            - 0 represents an empty cell
            - Numbers 1-9 represent filled cells
    
    Returns:
        list: A list of tuples (i,j) containing the coordinates of all non-zero cells, where:
            - i: row index (0-8)
            - j: column index (0-8)
            - Each tuple represents a position that is already filled
    
    Example:
        For a board where only position (0,0) contains 5 and (1,1) contains 3:
        >>> board = [[5,0,0...], [0,3,0...], ...]
        >>> coords_already_full(board)
        [(0,0), (1,1)]
    
    Note:
        - Used to identify positions that should not be modified during puzzle solving
        - Returns empty list if board is completely empty (all zeros)
        - Order of coordinates in returned list follows row-major order
    """
    list = []
    
    for i in range(9):
        for j in range(9):
            if board[i][j] != 0:
                list.append((i,j))
    
    return list


def fitness(board):
    """
    Calculates the fitness score for a Sudoku board by checking for duplicates in columns and 3x3 squares.
    The fitness score is negative and represents how many rule violations are present - a perfect valid board has score 0.

    Args:
        board (list): A 9x9 2D list representing the Sudoku board, where each cell contains a number from 1 to 9

    Returns:
        int: A fitness score <= 0, where:
            - 0 represents no duplicates in columns and 3x3 squares
            - Negative values indicate the total number of duplicate violations
            - Each duplicate number in a column or 3x3 square decrements the score by (count - 1)

    Example:
        If a column contains [1,1,1,2,3,4,5,6,7], the number 1 appears 3 times, 
        so it will contribute -2 to the fitness score (3-1 = 2 violations)

    Note:
        - The function only checks columns and 3x3 squares, not rows
        - Multiple occurrences of the same number in a column or 3x3 square are penalized
        - The more duplicates present, the lower (more negative) the fitness score
    """
    
    fitness_score = 0
    
    for j in range(9): #for each column
        seen = {}
        for i in range(9): # Check each cell in the column
            if board[i][j] in seen:
                seen[board[i][j]] += 1
            else:
                seen[board[i][j]] = 1
        for key in seen: # Subtract fitness for repeated numbers
            fitness_score -= (seen[key] - 1)
    
    for m in range(3): # For each 3x3 square
        for n in range(3):
            seen = {}
            for i in range(3 * n, 3 * (n + 1)):  # Check cells in 3x3 square
                for j in range(3 * m, 3 * (m + 1)):
                    if board[i][j] in seen:
                        seen[board[i][j]] += 1
                    else:
                        seen[board[i][j]] = 1
            for key in seen: # Subtract fitness for repeated numbers
                fitness_score -= (seen[key] - 1)
    
    return fitness_score


def roulette_wheel_selection2(population, already_parent = None):
    """
    Implements roulette wheel selection method for genetic algorithms.
    This version prevents selecting the same parent twice.

    Args:
        population (list): List of tuples (individual, fitness) where:
            - individual: represents a single individual in the population
            - fitness: numerical value representing the individual's goodness (lower values are better)
        already_parent (tuple, optional): Tuple (individual, fitness) representing an already selected parent
            to exclude from selection. Defaults to None.

    Returns:
        tuple: A tuple (individual, fitness) selected from the population using roulette wheel method.
            Selection probability is proportional to the fitness value.

    Note:
        - Prevents selection of the same individual as parent using np.array_equal
    """
    
    # Calculate total fitness sum of the population
    total_fitness = -sum(individual[1] for individual in population)
    
    if already_parent is not None:
        total_fitness += already_parent[1]

    # Generate a random number between 0 and the total fitness sum
    spin = random.uniform(0, total_fitness)

    # Iterate through the population and sum fitness values until the spin value is reached
    fitness_sum = 0
    for individual in population:
        # doesn't take 2 equal parents
        if not np.array_equal(individual, already_parent):
            fitness_sum += -individual[1]
            if fitness_sum >= spin:
                return individual
            
    
def selection(population, already_parent = None):
    """
    Performs random selection of an individual from a population, with the option to prevent 
    selecting an already selected parent.
    
    Args:
        population (list): List of tuples (individual, fitness) where:
            - individual: represents a single individual in the population
            - fitness: numerical value representing the individual's goodness
        already_parent (tuple, optional): Tuple (individual, fitness) representing a previously 
            selected parent to be excluded from selection. Defaults to None.
    
    Returns:
        tuple: A randomly selected (individual, fitness) tuple from the population, 
        guaranteed to be different from already_parent if specified.
    
    Process:
        1. Continuously generates random indices until a valid selection is made
        2. If already_parent is specified, ensures selected individual is different
        3. Returns the selected individual once conditions are met
    
    Note:
        - Uses uniform random selection (all individuals have equal probability)
        - Comparison between individuals is done using numpy's array_equal for deep comparison
    """
    
    while True:
        rand = random.randint(0, len(population) - 1)
        # if the second parent is not equal to the first selected
        if already_parent is not None:
            if not np.array_equal(population[rand][0], already_parent):
                break
        else:
            break
    
    return population[rand]
    
    
def crossover2(parent1, parent2):
    """
    Performs single-point crossover between two parent Sudoku boards to create a child board.
    Uses row-wise crossover where entire rows are swapped at the crossover point.
    
    Args:
        parent1 (list): First parent board - a 9x9 2D list representing a complete Sudoku board
        parent2 (list): Second parent board - a 9x9 2D list representing a complete Sudoku board
    
    Returns:
        list: A new 9x9 2D list representing the child board where:
            - Rows 0 to crossover_point-1 come from parent1
            - Rows crossover_point to 8 come from parent2
    
    Process:
        1. Randomly selects a crossover point between rows 1 and 8
        2. Creates a deep copy of parent2 as the base for the child
        3. Copies rows from parent1 up to the crossover point into the child
    
    Example:
        If crossover_point = 3:
        - Child rows 0,1,2 will be from parent1
        - Child rows 3,4,5,6,7,8 will be from parent2
    
    Note:
        - Creates new copies of rows to prevent reference issues
        - Preserves the integrity of individual rows
        - Does not guarantee valid Sudoku constraints in the resulting child
    """
    
    crossover_point = random.randint(1, 8)
    
    child = copy.deepcopy(parent2)
    
    for i in range(crossover_point):
        child[i] = parent1[i].copy()
            
    return child


def crossover(parent1, parent2):
    """
    Performs uniform crossover between two parent Sudoku boards to create a child board.
    Each row has a 50% chance of being inherited from either parent.
    
    Args:
        parent1 (list): First parent board - a 9x9 2D list representing a complete Sudoku board
        parent2 (list): Second parent board - a 9x9 2D list representing a complete Sudoku board
    
    Returns:
        list: A new 9x9 2D list representing the child board.
    
    Process:
        1. Creates a deep copy of parent2 as the base for the child
        2. For each row (0-8):
            - Generates random number between 0 and 1
            - If random < 0.5: copies row from parent1
            - If random >= 0.5: keeps row from parent2 (already copied)
    
    Example:
        A possible outcome might be:
        - Rows 0,3,5,7 from parent1
        - Rows 1,2,4,6,8 from parent2
        (actual inheritance pattern is random)
    
    Note:
        - More diverse offspring compared to single-point crossover
        - Creates new copies of rows to prevent reference issues
        - Preserves the integrity of individual rows
        - Does not guarantee valid Sudoku constraints in the resulting child
    """
    child = copy.deepcopy(parent2)
    
    for id, row in enumerate(child):
        if random.random() < 0.5:
             child[id] = parent1[id].copy() 
            
    return child
        
        
def mutate(board, already_filled, rate):
    """
    Performs mutation on a Sudoku board by randomly swapping numbers within rows, 
    respecting the positions that were initially filled.
    
    Args:
        board (list): A 9x9 2D list representing a complete Sudoku board
        already_filled (list): List of tuples (row,col) representing positions that were 
            initially filled and should not be modified
        rate (float): Mutation rate between 0 and 1, representing the probability of 
            mutating each row
    
    Returns:
        list: The mutated 9x9 Sudoku board. The board is modified in-place and also returned.
    
    Process:
        1. For each row in the board:
            - Generates random number to determine if row should mutate (based on rate)
            - If mutation occurs:
                a. Randomly selects two different positions in the row
                b. Checks if both positions are not in already_filled list
                c. If valid positions found, swaps the numbers in these positions
                d. Repeats position selection if chosen positions are invalid
    
    Example:
        If rate = 0.1 and row 3 is selected for mutation:
        - Original row: [1,2,3,4,5,6,7,8,9]
        - After swapping positions 2 and 5: [1,2,6,4,5,3,7,8,9]
    
    Note:
        - Only swaps numbers within the same row, preserving row-wise uniqueness
        - Will keep trying to find valid positions if initial selections are invalid
        - Higher mutation rates increase genetic diversity but may slow convergence
        - Respects the initial puzzle constraints (already_filled positions)
    """
    
    for id, row in enumerate(board):
        if random.random() < rate:
            while True:
                pos1 = random.randint(0, 8)
                pos2 = random.randint(0, 8)
                if not (id, pos1) in already_filled and not (id, pos2) in already_filled and pos1 != pos2:
                    row[pos1], row[pos2] = row[pos2], row[pos1]
                    break
    
    return board

In [3]:
# Example Board
list_board = [
("""
.7. ..4 13.
... 2.7 ..6
..5 .13 .2.
..1 ..2 ...
..2 19. .57
..3 .45 8.2
.1. 378 26.
367 ... 58.
8.9 ..1 .7.
""","easy1"),

("""
37. 5.. ..6
... 36. .12
... .91 75.
... 154 .7.
..3 .7. 6..
.5. 638 ...
.64 98. ...
59. .26 ...
2.. ..5 .64
""", "easy2"),

    ("""
    ..7..249.
    .....52..
    284.167..
    .49..382.
    ...5....9
    .72.94.31
    ..57389.2
    .6....1..
    .2.6.13.4
    """, "easy3"),

    ("""
    ..615...2
    ......7..
    7...84...
    3..9.8561
    .8.......
    ..2..5..8
    ..7...6.3
    53.6..984
    .4...1..7
    """, "medium1"),

    ("""
    ..1.....8
    .4.......
    82754....
    ...41..87
    ...7.3.92
    .7..285.6
    49......3
    ..3....69
    218......
    """, "medium2"),

    ("""
    71.53.482
    25...4...
    86.97213.
    1..36....
    ..2...6..
    ....91.43
    3....9...
    ..17...26
    4.7...35.
    """, "medium3"),

    ("""
    8.6...3..
    .4.1.2.5.
    .925.....
    ....13.4.
    ....5..63
    42..87...
    ..9...781
    .......39
    ..489.5..
    """, "hard1"),

    ("""
    .....253.
    61.....2.
    ...3..649
    .....7...
    .45....18
    .9682...3
    3...45.6.
    4..67....
    ..1.9...4
    """, "hard2"),

("""
8........
..36.....
.7..9.2..
.5...7...
....45.7.
...1...3.
..1....68
..85...1.
.9....4..
""", "hard3"),

    ("""
     .........
     .....3.85 
     ..1.2....
     ...5.7...
     ..4...1..
     .9.......
     5......73
     ..2.1....
     ....4...9
    """, "very hard")
]

In [5]:
import time
def genetic_algorithm(entry_board, pop_size, max_generations, mutation_rate, elitism_percentage):
    """
    Implements a genetic algorithm to solve a Sudoku puzzle.
    
    Args:
        entry_board (str): String representation of the initial Sudoku board
        pop_size (int): Size of the population in each generation
        max_generations (int): Maximum number of generations to run
        mutation_rate (float): Probability of mutation (0-1) for each row
        elitism_percentage (float): Percentage of best individuals to preserve (0-1)
    
    Returns:
        tuple: (best_board_state, execution_time, generations_needed) where:
            - best_board_state (list): 9x9 2D list of the best solution found
            - execution_time (float): Total processing time in seconds
            - generations_needed (int): Number of generations passed until termination
    
    Process:
        1. Initialization:
            - Parses input board and identifies fixed positions
            - Generates initial population of valid board states
            - Sets up parallel processing parameters
    
        2. Main Loop (for each generation):
            a. Evaluates fitness of all individuals
            b. Checks for perfect solution (fitness = 0)
            c. Implements elitism by preserving best individuals
            d. Creates new population through:
                - Selection of parents
                - Crossover to create children
                - Mutation of children
                - Ensures population diversity
    
        3. Termination:
            - When perfect solution is found
            - When max_generations is reached
    
    
    Note:
        - Implements diversity preservation in new population
        - Maintains fixed cells from original puzzle
        - Early termination on perfect solution
        - Solution quality depends on parameter tuning
        - Can run for max_generations without finding perfect solution
    """
    board = parse_board(entry_board)

    # Generate the initial population
    population = [(generate_board_state(board), 0) for _ in range(pop_size)]
    
    # save in a var the coordinates to the cells already filled
    already_filled = coords_already_full(board)
    
    # set the best board to the initial board
    best_board_state = board
    
    i = 0
    start_time = time.process_time()
    # Main Genetic Algorithm loop
    for generation in range(max_generations):
        
        # Create the next generation
        new_population = []
        
        # Calculate fitness for each board state
        population = [(board_state, fitness(board_state)) for board_state, _ in population]
        
        random.shuffle(population)
        
        # Check if solution is found
        best_board_state = max(population, key=lambda x: x[1])[0]
        if fitness(best_board_state) == 0:
            print("Solution found in generation", generation)
            return best_board_state, time.process_time() - start_time, generation
            
        ### PRINT STATE
        #print("Generazione ", generation)
        #print("Fitness: ", max(population, key=lambda x: x[1])[1])
        #print_board(best_board_state)
        #print()
    
        population_copy = population.copy()
        # Elitism: Keep the best board state from the previous generation
        for n in range(int(elitism_percentage * pop_size)):
            
            max_individual = max(population_copy, key=lambda x: x[1])
            if not any(np.array_equal(max_individual[0], existing[0]) for existing in new_population):
                new_population.append(max_individual)
            population_copy = [x for x in population_copy if not np.array_equal(x[0], max_individual[0])]
            
        # Perform selection, crossover, and mutation
        while len(new_population) < pop_size:
            parent1 = selection(population)
            # Select the other parent without the chances that can take the parent1
            parent2 = selection(population, parent1)
            
            child = crossover(parent1[0], parent2[0])
            
            child = mutate(child, already_filled, mutation_rate)
            
            # Optimization: insert only different childs
            if not any(np.array_equal(child, existing[0]) for existing in new_population):
                new_population.append((child, 0))
    
        # Update the population
        population = new_population
        i += 1
        
    # returns the best solution
    return best_board_state, time.process_time() - start_time, max_generations

In [7]:
def run_single_test(board_state):
    return genetic_algorithm(
        entry_board=board_state, 
        pop_size=100, 
        max_generations=1000, 
        mutation_rate=0.1, 
        elitism_percentage=0.2
    )

# Numbers of cores used for the test
NUM_CORE = 7

# id of the board to use for the tests
id_board = 9

# Prepare the list to give to parallel processes
num_tests = 30
board_states = [list_board[id_board][0] for _ in range(num_tests)]

# Parallelization
pool = Pool(NUM_CORE)
results = pool.map(run_single_test, board_states)
pool.close()
pool.join()

# results array from the executions
solutions_array = []
execution_time_array = []
generations_array = []

for solution, exec_time, generations in results:
    solutions_array.append(solution)
    execution_time_array.append(exec_time)
    generations_array.append(generations)
    
    
average_time = sum(execution_time_array) / len(solutions_array)

sum_generations = 0
solved_quantity = 0
sum_collisions = 0

for index, board in enumerate(solutions_array):
    if board_is_solved(board):
        sum_generations += generations_array[index]
        solved_quantity +=1
    else:
        sum_collisions += - fitness(board)
       
print("Average execution time: ", average_time, "difficulty: ", list_board[id_board][1]) 
print("Number of execution solved: ", solved_quantity, "/", num_tests)
print("The average number of generations to find a solution (when found) is: ", sum_generations / solved_quantity if solved_quantity > 0 else 0)
print("The average number of collisions found (so when board is not solved) is: ", sum_collisions / (num_tests - solved_quantity))

finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
finito
Average execution time:  223.32395833333334 difficulty:  very hard
Number of execution solved:  0 / 30
The average number of generations to find a solution (when found) is:  0
The average number of collisions found (so when board is not solved) is:  5.8
